In [84]:
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest
from statsmodels.stats.proportion import proportions_ztest

from scipy.stats import ttest_ind, norm, t, ttest_1samp
from scipy.stats import f_oneway
from scipy.stats import sem

In [85]:
df = pd.read_csv('./data/Student_performance_data.csv')
df

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,3388,18,1,0,3,10.680555,2,0,4,1,0,0,0,3.455509,0.0
2388,3389,17,0,0,1,7.583217,4,1,4,0,1,0,0,3.279150,4.0
2389,3390,16,1,0,2,6.805500,20,0,2,0,0,0,1,1.142333,2.0
2390,3391,16,1,1,0,12.416653,17,0,2,0,1,1,0,1.803297,1.0


### **Uso de 'GPA' como Variable Objetivo**

'GPA' vs. 'GradeClass':

**'GPA'**: El GPA (Grade Point Average) es una medida continua que refleja el promedio de calificaciones de un estudiante en una escala numérica. Es una representación más precisa y granular del rendimiento académico del estudiante.
**'GradeClass'**: Esta variable, si bien también mide el rendimiento académico, lo hace de manera categórica, agrupando a los estudiantes en clases o rangos de calificaciones.

Razón para elegir 'GPA':

* Al usar el GPA, obtenemos una medida continua que permite una mayor precisión en los análisis estadísticos. Las diferencias en el rendimiento académico se pueden capturar con mayor exactitud que con una variable categórica.

* Muchas de las pruebas estadísticas empleadas (como las correlaciones de Pearson y Spearman, y la prueba t) funcionan mejor con datos continuos y proporcionan resultados mas significativos.

## **<u> Planteamiento de hipotesis </u>**

### **1. En relación a la variable StudyTimeWeekly**
* **H0**: No hay una relación de proporcionalidad directa entre el tiempo semanal de estudio y las notas de los alumnos (cuanto mas estudio, mejores notas)
* **H1**: Hay una relación de proporcionalidad directa entre el tiempo semanal de estudio y las notas de los alumnos (cuanto mas estudio, mejores notas)

In [86]:
# Prueba de correlación de Pearson (StudyTimeWeekly-GPA)

corr, p_value = pearsonr(df['StudyTimeWeekly'], df['GPA'])

print(f'Correlación de Pearson: {corr:.3f}, p_value: {p_value:.3f}')

if p_value < 0.05:
    print("H1 <> Hay una relación significativa entre el tiempo semanal de estudio y las notas.")
else:
    print("H0 <> No hay una relación significativa entre el tiempo semanal de estudio y las notas.")

Correlación de Pearson: 0.179, p_value: 0.000
H1 <> Hay una relación significativa entre el tiempo semanal de estudio y las notas.


### **2. En relación a la variable Absences**
* **H0**: No hay una relación de proporcionalidad inversa entre el numero de ausencias las notas de los alumnos (cuanto mas absencias, peores notas)
* **H1**: Hay una relación de proporcionalidad inversa entre el numero de ausencias las notas de los alumnos (cuanto mas absencias, peores notas)

In [87]:
# Prueba de correlación de Spearman (Absences-GPA)

corr, p_value = spearmanr(df['Absences'], df['GPA'])

print(f'Correlación de Spearman: {corr:.3f}, p_value: {p_value:.3f}')
if p_value < 0.05:
    print("H1 <> Hay una relación significativa entre el número de ausencias y las notas.")
else:
    print("H0 <> No hay una relación significativa entre el número de ausencias y las notas.")

Correlación de Spearman: -0.925, p_value: 0.000
H1 <> Hay una relación significativa entre el número de ausencias y las notas.


### **3. En relación a la variable Gender**
* **H0**: No hay diferencias significativas en las notas de hombres y mujeres
* **H1**: Hay diferencias significativas en las notas de hombres y mujeres

In [88]:
# Prueba t de muestras independientes (Gender-GPA)

# Dividir en grupos por género
df_male = df[df['Gender'] == 0]['GPA']
df_female = df[df['Gender'] == 1]['GPA']

# Prueba t de muestras independientes
t_stat, p_value = ttest_ind(df_male, df_female, equal_var=False)

print(f't_stat: {t_stat:.3f}, p_value: {p_value:.3f}')
if p_value < 0.05:
    print("H1 <> Hay diferencias significativas en las notas entre hombres y mujeres.")
else:
    print("H0 <> No hay diferencias significativas en las notas entre hombres y mujeres.")

t_stat: 0.653, p_value: 0.514
H0 <> No hay diferencias significativas en las notas entre hombres y mujeres.


### **4. En relación a la variable Ethnicity**
* **H0**: No hay diferencias significativas en las notas alumnos de diferentes etnias
* **H1**: Hay diferencias significativas en las notas alumnos de diferentes etnias

In [89]:
# Prueba ANOVA de una vía

# Dividir en grupos por etnicidad
ethnic_groups = df['Ethnicity'].unique()
groups = [df[df['Ethnicity'] == group]['GPA'] for group in ethnic_groups]

# ANOVA de una vía
f_stat, p_value = f_oneway(*groups)

print(f'f_stat: {f_stat:.3f}, p_value: {p_value:.3f}')
if p_value < 0.05:
    print("H1 <> Hay diferencias significativas en las notas entre diferentes etnias.")
else:
    print("H0 <> No hay diferencias significativas en las notas entre diferentes etnias.")

f_stat: 0.958, p_value: 0.412
H0 <> No hay diferencias significativas en las notas entre diferentes etnias.


### **5. En relación a la variable ParentalEducation**
* **H0**: No hay diferencias significativas en las notas de los alumnos según el nuvel de educación parental
* **H1**: Hay diferencias significativas en las notas de los alumnos según el nuvel de educación parental

In [90]:
# Prueba ANOVA de una vía

# Dividir en grupos por educación parental
education_levels = df['ParentalEducation'].unique()
groups = [df[df['ParentalEducation'] == level]['GPA'] for level in education_levels]

# ANOVA de una vía
f_stat, p_value = f_oneway(*groups)
print(f'f_stat: {f_stat:.3f}, p_value: {p_value:.3f}')

if p_value < 0.05:
    print("H1 <> Existen diferencias significativas en las notas según el nivel de educación parental.")
else:
    print("H0 <> No hay evidencia suficiente de diferencias significativas en las notas según el nivel de educación parental.")

f_stat: 1.808, p_value: 0.124
H0 <> No hay evidencia suficiente de diferencias significativas en las notas según el nivel de educación parental.


### **6. En relación a la variable Tutoring**
* **H0**: No hay diferencias significativas en las notas de los alumnos que tienen tutor y los que no
* **H1**: Hay diferencias significativas en las notas de los alumnos que tienen tutor y los que no

In [91]:
# Prueba t de muestras independientes (Tutoring-GPA)

# Dividir en grupos por tutoría
df_tutored = df[df['Tutoring'] == 1]['GPA']
df_no_tutored = df[df['Tutoring'] == 0]['GPA']

# Prueba t de muestras independientes
t_stat, p_value = ttest_ind(df_tutored, df_no_tutored, equal_var=False)

print(f't_stat: {t_stat:.3f}, p_value: {p_value:.3f}')
if p_value < 0.05:
    print("H1 <> Los alumnos con tutor tienen significativamente mejores notas.")
else:
    print("H0 <> No hay diferencias significativas en las notas entre los alumnos con tutoría y los que no.")

t_stat: 7.172, p_value: 0.000
H1 <> Los alumnos con tutor tienen significativamente mejores notas.


### **7. En relación a la variable ParentalSupport**
* **H0**: No hay una relación significativa entre el soporte parental y las notas
* **H1**: Hay una relación significativa entre el soporte parental y las notas

In [92]:
# Prueba de correlación de Pearson (ParentalSupport-GPA)

corr, p_value = pearsonr(df['ParentalSupport'], df['GPA'])
print(f'Correlación de Pearson: {corr:.3f}, p_value: {p_value:.3f}')

if p_value < 0.05:
    print("H1 <> Existe una relación significativa entre el soporte parental y las notas.")
else:
    print("H0 <> No hay evidencia suficiente de una relación significativa entre el soporte parental y las notas.")

Correlación de Pearson: 0.191, p_value: 0.000
H1 <> Existe una relación significativa entre el soporte parental y las notas.


### **8. En relación a la variable Extracurricular**
* **H0**: No hay diferencias significativas en las notas de los alumnos que hacen actividades extracurriculares y los que no
* **H1**: Hay diferencias significativas en las notas de los alumnos que hacen actividades extracurriculares y los que no

In [93]:
# Prueba t de muestras independientes (Extracurricular-GPA)

extracurricular_grades = df[df['Extracurricular'] == 1]['GPA']
no_extracurricular_grades = df[df['Extracurricular'] == 0]['GPA']

t_stat, p_value = ttest_ind(extracurricular_grades, no_extracurricular_grades, equal_var=False)
print(f't_stat: {t_stat:.3f}, p_value: {p_value:.3f}')

if p_value < 0.05:
    print("H1 <> Existen diferencias significativas en las notas entre los alumnos que hacen actividades extracurriculares y los que no.")
else:
    print("H0 <> No hay evidencia suficiente de diferencias significativas en las notas entre los alumnos que hacen actividades extracurriculares y los que no.")

t_stat: 4.609, p_value: 0.000
H1 <> Existen diferencias significativas en las notas entre los alumnos que hacen actividades extracurriculares y los que no.


### **9. En relación a la variable Sports**
* **H0**: No hay diferencias significativas en las notas de los alumnos que practican deporte y los que no
* **H1**: Hay diferencias significativas en las notas de los alumnos que practican deporte y los que no

In [94]:
# Prueba t de muestras independientes (Sports-GPA)

sports_grades = df[df['Sports'] == 1]['GPA']
no_sports_grades = df[df['Sports'] == 0]['GPA']

t_stat, p_value = ttest_ind(sports_grades, no_sports_grades, equal_var=False)
print(f't_stat: {t_stat:.3f}, p_value: {p_value:.3f}')

if p_value < 0.05:
    print("H1 <> Existen diferencias significativas en las notas entre los alumnos que practican deporte y los que no.")
else:
    print("H0 <> No hay evidencia suficiente de diferencias significativas en las notas entre los alumnos que practican deporte y los que no.")

t_stat: 2.851, p_value: 0.004
H1 <> Existen diferencias significativas en las notas entre los alumnos que practican deporte y los que no.


### **10. En relación a la variable Music**
* **H0**: No hay diferencias significativas en las notas de los alumnos que practican musica y los que no
* **H1**: Hay diferencias significativas en las notas de los alumnos que practican musica y los que no

In [95]:
# Prueba t de muestras independientes (Music-GPA)

music_grades = df[df['Music'] == 1]['GPA']
no_music_grades = df[df['Music'] == 0]['GPA']

t_stat, p_value = ttest_ind(music_grades, no_music_grades, equal_var=False)
print(f't_stat: {t_stat:.3f}, p_value: {p_value:.3f}')

if p_value < 0.05:
    print("H1 <> Existen diferencias significativas en las notas entre los alumnos que practican música y los que no.")
else:
    print("H0 <> No hay evidencia suficiente de diferencias significativas en las notas entre los alumnos que practican música y los que no.")

t_stat: 3.597, p_value: 0.000
H1 <> Existen diferencias significativas en las notas entre los alumnos que practican música y los que no.


### **11. En relación a la variable Volunteering**
* **H0**: No hay diferencias significativas en las notas de los alumnos que hacen voluntariado y los que no
* **H1**: Hay diferencias significativas en las notas de los alumnos que hacen voluntariado y los que no

In [96]:
# Prueba t de muestras independientes (Volunteering-GPA)

volunteering_grades = df[df['Volunteering'] == 1]['GPA']
no_volunteering_grades = df[df['Volunteering'] == 0]['GPA']

t_stat, p_value = ttest_ind(volunteering_grades, no_volunteering_grades, equal_var=False)
print(f't_stat: {t_stat:.3f}, p_value: {p_value:.3f}')

if p_value < 0.05:
    print("H1 <> Existen diferencias significativas en las notas entre los alumnos que hacen voluntariado y los que no.")
else:
    print("H0 <> No hay evidencia suficiente de diferencias significativas en las notas entre los alumnos que hacen voluntariado y los que no.")

t_stat: 0.161, p_value: 0.872
H0 <> No hay evidencia suficiente de diferencias significativas en las notas entre los alumnos que hacen voluntariado y los que no.
